In [84]:
import numpy as np
import pyvista as pv
import os

In [85]:
# Define mesh path
case_path = '/home/dana/Documents/SimVascular/svMultiPhysics/NoisyPipe/NP_0042/'
mesh_path = case_path + 'mesh/'
geom_path = case_path + 'geom/'
pp_path = case_path + 'data/'
pref = '4-procs/result'
data_path = case_path + pref + '_100.vtu'

# Load mesh
f_vol = pv.read(mesh_path + "fluid/mesh-complete.vtu")
spline = np.load(geom_path + "spline.npy")
contours = np.load(geom_path + "contours.npy")
mesh = pv.read(data_path)

In [ ]:
# Create bounding box
bbox = pv.Box((-4.5,4.5,-1.5,1.5,-1.5,1.5))
cell_dimensions = np.array((100,20,20))
bbox_v = bbox.voxelize(dimensions=cell_dimensions + 1)
# bbox = mesh.voxelize(dimensions=cell_dimensions + 1)

# Compute implicit distance
bbox_d = bbox_v.compute_implicit_distance(mesh.extract_surface())
# bbox_d['implicit_distance']*=-1
bbox_d = bbox_d.point_data_to_cell_data()
# bbox_m = bbox_d + mesh
# bbox_m = bbox_m.voxelize_binary_mask(dimensions=cell_dimensions + 1,
                                    #  background_value=1,
                                    #  foreground_value=0)
bbox_m = bbox_d.copy()
bbox_m.cell_data['mask'] = bbox_d.cell_data['implicit_distance']>0
bbox_m = bbox_m.point_data_to_cell_data()

b_clip = bbox_d.clip(normal='y',origin=[0,0,0])
b_m_clip = bbox_m.clip(normal='y',origin=[0,0,0])

dist = bbox_d.cell_data['implicit_distance']
mask = bbox_m.cell_data['mask']

# Interpolate results onto query points
data_r = bbox_v.interpolate(mesh)

# Plot
pl = pv.Plotter(shape=(1,2))
pl.subplot(0,0)
# pl.add_mesh(bbox,opacity=0.3,color='lightblue')
# pl.add_mesh(f_vol, style='wireframe',color='lightblue')
pl.add_mesh(b_clip,scalars='implicit_distance',cmap='bwr',clim=[-1,1])
# pl.add_points(spline,color='red')
# for contour in contours:
#     pl.add_points(contour,color='red')
pl.camera_position='zx'
pl.subplot(0,1)
# pl.add_mesh(f_vol, style='wireframe',color='lightblue',opacity=0.3)
pl.add_mesh(b_m_clip,scalars='mask',interpolate_before_map=False,cmap='bwr')
pl.camera_position='zx'
pl.show()

Widget(value='<iframe src="http://localhost:33577/index.html?ui=P_0x7b09987059d0_17&reconnect=auto" class="pyv…

: 

In [65]:
points/max_val

pyvista_ndarray([[-1.,  0.,  0.],
                 [ 1.,  0.,  0.],
                 [ 0., -1.,  0.],
                 [ 0.,  1.,  0.],
                 [ 0.,  0., -1.],
                 [ 0.,  0.,  1.]])

In [70]:

bbox_n = bbox.copy()
points = bbox.points
max_val = np.max(points,axis=0)
min_val = np.min(points,axis=0)
bbox_n.points = (points-min_val)/(max_val-min_val) # Normalize to (0,1)
# bbox_n.points = points/max_val # Normalize to (0,1)

bbox_n.plot()
bbox_n.cell_centers()

Widget(value='<iframe src="http://localhost:33577/index.html?ui=P_0x7b0a93e39010_8&reconnect=auto" class="pyvi…

PolyData (0x7b09931d1ba0)
  N Cells:    6
  N Points:   6
  N Strips:   0
  X Bounds:   0.000e+00, 1.000e+00
  Y Bounds:   0.000e+00, 1.000e+00
  Z Bounds:   0.000e+00, 1.000e+00
  N Arrays:   0

In [ ]:
data_pv = data_r.copy()

scalar_names = ['Pressure', 'Divergence', 'magU', 'u', 'v', 'w', 'magW', 'magWSS']

# Pressure
data_pv['Pressure'] = data_pv.point_data['Pressure']

# Divergence
data_pv['Divergence'] = data_pv.point_data['Divergence']

# Magnitude of velocity
u = data_pv.point_data['Velocity']
magU = np.linalg.norm(u,axis=1)
data_pv['magU'] = magU

# u, v, w
data_pv['u'] = u[:,0]
data_pv['v'] = u[:,1]
data_pv['w'] = u[:,2]

# Magnitude of vorticity
w = data_pv.point_data['Vorticity']
magW = np.linalg.norm(w,axis=1)
data_pv['magW'] = magW

# Magnitude of WSS
WSS = data_pv.point_data['WSS']
magWSS = np.linalg.norm(WSS,axis=1)
data_pv['magWSS'] = magWSS

data_pv = data_pv.point_data_to_cell_data()

for scalar in scalar_names:
    data_pv.cell_data[scalar][mask] = np.nan

In [220]:
# Plot data
pl = pv.Plotter()
pl.add_mesh(data_pv.clip(normal='y',origin=[0,0,0]),scalars = 'magU',clim=[0,20])
pl.camera_position='zx'
pl.show()

Widget(value='<iframe src="http://localhost:45083/index.html?ui=P_0x76514f9755b0_104&reconnect=auto" class="py…

In [ ]:
# Assemble coords array
dist = dist.reshape((-1,1))
mask = mask.reshape((-1,1))
coords = np.hstack([bbox_v.cell_centers().points,dist,mask])

# Assemble data array
data = np.dstack([data_pv.cell_data['u'],#.reshape((-1,1)),
                    data_pv.cell_data['v'],#.reshape((-1,1)),
                    data_pv.cell_data['w'],#.reshape((-1,1)),
                    data_pv.cell_data['Pressure']#.reshape((-1,1))
                    ])

# Create directories if they don't exist
# os.makedirs(coords_path, exist_ok=True)
# np.save(coords_path + 'coords.npy',coords)

coords.shape

(65536, 5)

In [51]:
num_geoms = 40
geom_ids = np.arange(num_geoms)
geom_ids = np.repeat(geom_ids,data.shape[0]//num_geoms)
# geom_ids = np.reshape(geom_ids,(-1,))
# geom_ids = np.repeat(geom_ids,data.shape[1],axis=1)
geom_ids.shape
test=torch.tensor([1])
test.get_device()

-1

In [83]:
data = np.load('postprocessed_data/NP_data/data.npy')
coords = np.load('postprocessed_data/NP_data/coords.npy')
coords_o = np.load('postprocessed_data/NP_data/coords_o.npy')
sdf = np.load('postprocessed_data/NP_data/sdf.npy')
sdf_o = np.load('postprocessed_data/NP_data/sdf_o.npy')
sdf_o = np.squeeze(sdf_o)

tidx = 20
data_o = data[tidx,:,:] # original

# Reshape for StructuredGrid
mesh_d = pv.StructuredGrid()
mesh_d.points = coords_o
mesh_d.dimensions = (100, 20, 20)  
mesh_o = mesh_d.copy()

# Add scalar field
mesh_o['u'] = data_o[:,0]
mesh_o['v'] = data_o[:,1]
mesh_o['w'] = data_o[:,2]
mesh_o['p'] = data_o[:,3]
mesh_o['p'] = mesh_o['p']/np.maximum(mesh_o['p'],5e4)
pmin = np.min(mesh_o['p'][mesh_o['p']!=0])
pmax = np.max(mesh_o['p'][mesh_o['p']!=0])
mesh_o['g'] = sdf_o
mesh_o['m'] = sdf_o>0
mesh_o = mesh_o.clip('y')

# Define scalar and value ranges
scalar = 'p'
clim_dict = {
    'u': (0,10),
    'v': (0,1),
    'w': (0,1),
    'p': (pmin,pmax),
    'g': (-1,1),
    'm': (0,1),
}

# Visualize
pl = pv.Plotter()
pl.add_mesh(mesh_o, scalars=scalar,
            clim=clim_dict[scalar]
            # show_edges=True
            )
pl.camera_position='zx'
pl.show()


Widget(value='<iframe src="http://localhost:33577/index.html?ui=P_0x7b0990964650_16&reconnect=auto" class="pyv…